In [1]:
!pip3 install neattext
import pandas as pd
import neattext.functions as nfx
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# !mkdir ~/.kaggle
# !touch ~/.kaggle/kaggle.json

# api_token = api_token = {"username":"<Username>","key":"<API_KEY>"}

# import json

# with open('/root/.kaggle/kaggle.json', 'w') as file:
#     json.dump(api_token, file)

# !chmod 600 ~/.kaggle/kaggle.json

In [3]:
# ! kaggle datasets download -d thedevastator/udemy-courses-revenue-generation-and-course-anal
# !unzip udemy-courses-revenue-generation-and-course-anal.zip

In [ ]:
pip install kaggle


In [ ]:
business_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-business-courses.csv")
design_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-design-courses.csv")
music_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-music-courses.csv")
web_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-web-development.csv")
entry_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\Entry Level Project Sheet - 3.1-data-sheet-udemy-courses-web-development.csv")
df = pd.concat([business_df, design_df,music_df,web_df,entry_df])
df.head()

In [ ]:
 df['course_title'].iloc[1:5]

**Remove Stopwords**


**Remove special characters**

In [ ]:
df = df.drop_duplicates(subset=['course_title'])

df['clean_title'] = df['course_title'].astype(str)

df['clean_title'] = df['clean_title'].apply(nfx.remove_stopwords)

df['clean_title'] = df['clean_title'].apply(nfx.remove_special_characters)

df['clean_title'].iloc[1:5]

In [ ]:
countvect = CountVectorizer()

cv_mat = countvect.fit_transform(df['clean_title'])

cv_mat

In [ ]:
df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names_out())

df_cv_words

In [ ]:
cosine_sim_mat = cosine_similarity(cv_mat)

In [ ]:
def recommend_course(title,numrec = 10):
    
    course_index = pd.Series( df.index, index=df['clean_title']).drop_duplicates()
    
    index = course_index[title]

    scores = list(enumerate(cosine_sim_mat [index]))
    
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    selected_course_index = [i[0] for i in sorted_scores[1:]]

    selected_course_score = [i[1] for i in sorted_scores[1:]]

    rec_df = df.iloc[selected_course_index]

    rec_df['Similarity_Score'] = selected_course_score

    final_recommended_courses = rec_df[[
        'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]

    return final_recommended_courses.head(numrec)




ans = recommend_course('Introduction Financial Modeling',20)

ans

In [ ]:
ans.to_json(orient ='index')

In [ ]:
class RecommendCourses:
  def __init__(self):
    #create a dataframe 
    business_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-business-courses.csv")
    design_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-design-courses.csv")
    music_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-music-courses.csv")
    web_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\3.1-data-sheet-udemy-courses-web-development.csv")
    entry_df = pd.read_csv(r"C:\Users\devip\Desktop\MST_project\Courserecommendation\training\datasets\Entry Level Project Sheet - 3.1-data-sheet-udemy-courses-web-development.csv")
    df = pd.concat([business_df, design_df,music_df,web_df,entry_df])

    #drop duplicates from the course_title column
    df = df.drop_duplicates(subset=['course_title'])
    # clean_title column to string from course_title
    df['clean_title'] = df['course_title'].astype(str)
    # clean_title remove stopwords
    df['clean_title'] = df['clean_title'].apply(nfx.remove_stopwords)
    # clean_title remove special characters
    df['clean_title'] = df['clean_title'].apply(nfx.remove_special_characters)

    # create CountVectorizer
    countvect = CountVectorizer()
    cv_mat = countvect.fit_transform(df['clean_title'])

    # create CV words
    df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names_out())
    self.df = df
    self.cosine_sim_mat = cosine_similarity(cv_mat)
    print("Data loaded Successfully")


  def recommend_course(self,title,numrec = 10):
    course_index = pd.Series( df.index, index=df['clean_title']).drop_duplicates()
    if title in course_index:
      index = course_index[title]
      scores = list(enumerate(cosine_sim_mat [index]))
      sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
      selected_course_index = [i[0] for i in sorted_scores[1:]]
      selected_course_score = [i[1] for i in sorted_scores[1:]]
      rec_df = df.iloc[selected_course_index]
      rec_df['Similarity_Score'] = selected_course_score
      final_recommended_courses = rec_df[[
          'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]
      return {"error":None,"data":final_recommended_courses.head(numrec).to_json(orient ='index')}
    else:
      return {"error": "course with title name "+title+" not found"}

recommendation = RecommendCourses()
recommendation.recommend_course('Introduction Financial Modeling',20)
